## Morning practical 2 day 3

Welcome to the second practical of today. Here, you'll start on the process of actually implementing backpropagation. First, preparation of some functions we will need.



In [ ]:
#run this cell to set things up
import ipywidgets as widgets, numpy as np, pandas as pd
from numpy.random import default_rng
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import math
import seaborn as sns
from IPython.display import display, Markdown
from scipy.optimize import fmin_bfgs

In [ ]:
#important functions
def mySigmoid(data):
    output = 1/(1+ np.exp(-data))
    return output

## Loading in the MNIST data

We're going to be working with the MNIST data again, though now in a slighly different format of 28\*28 = 784 pixel images, where each pixel has a brightness from 0 to 255. We change this brightness to a value between 0 and 1.

In [ ]:
import tensorflow as tf
from tensorflow import keras

#load-in taken from here: https://keras.io/examples/vision/mnist_convnet/
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


## Subsampling the training data

At this point, the data is in the format of 60.000 training samples, each consisting of a 28\*28 matrix of pixel intensities, with 1 channel (i.e. there's just 1 colour here, grey. In a colour image you would have 3 channels: red, green and blue intensities).

For a dense neural network that we'll be working with, we'll just convert these 28\*28 matrices into vectors with 784 entries. We'll also subsample the data: this amount of data is fine for optimised routines, but for our own implementation 20.000 training samples is _plenty_. 

In [ ]:
from numpy.random import default_rng
rng = default_rng(42)

print(np.unique(y_train, axis = 0, return_counts= True))
#the training set is not quite balanced, but let's keep the proportions it has nonetheless.

uniqueLabels = np.unique(y_train, axis = 0, return_counts= True)

#if to make sure you don't subsample again if you rerun this cell
if np.sum(uniqueLabels[1]) >=20010:
    listChoiceIndices = []
    for index, label in enumerate(uniqueLabels[0]):
        print("Subsetting for label: " + str(label))
        nToPick = np.ceil(1/3*uniqueLabels[1][index]).astype(int)
        indicesToPick = np.where(np.all(y_train == label, axis = 1))
        choiceIndices = rng.choice(indicesToPick[0], size = nToPick, replace = False)
        listChoiceIndices.append(choiceIndices)
    choiceIndices = [val for sublist in listChoiceIndices for val in sublist]

    x_train, y_train = x_train[choiceIndices], y_train[choiceIndices]
    #convert matrices into vectors

x_train_vectors = np.vstack([np.ravel(elem) for elem in x_train])
x_test_vectors  = np.vstack([np.ravel(elem) for elem in x_test])


## Implementing the cost function

We'll be working with the same network architecture as yesterday (although slightly more inputs): a Hidden Layer (HL) with 25 nodes, and 10 output neurons (predicting each of the 10 possible digits).
![NN](NeuralNetwork.png)


First things first: we'll need to calculate the cost at the end of the network. We'll use the categorical cross-entropy, whose formula is: <br>
![CategoricalCrossEntropyFormula](CategoricalCrossEntropy.png) <br> <br>

Let's make a function that, ultimately, will return the gradients for every weight in our 2-layer neural network. We might want to use `fmin_bfgs` or another minimisation function on the weights (rather than bog-standard gradient descent). For this reason, we will make the function take in a list of all the thetas of the network. That's what these methods want: a single list/array of gradients. We internally turn this array back into the matrices $\Theta_1$ and $\Theta_2$. No stress: that's done for you. Your job: reimplement forward propagation in the function below, then calculate the cost with categorical cross-entropy. So:

* Implement forward propagation in the function below. This is the same as yesterday.
* After forward propagation, calculate the mean cost (1/m * total cost) for all the forward-propagated samples. There's no looping involved!

Hints:
* Don't forget to use `mySigmoid()`.
* First experiment outside the function body before putting things into the function!
* With the seed we've set for our rng, you should get a mean cost of 6.84 for these random thetas.

In [ ]:
rng = default_rng(42)
thetaOneMatrix = rng.uniform(-0.12, 0.12, size = (25, 785))
thetaTwoMatrix = rng.uniform(-0.12, 0.12, size = (10, 26))

nnThetas       = np.append(np.ravel(thetaOneMatrix), np.ravel(thetaTwoMatrix))
inputLayerSize = 784
hiddenLayerSize = 25
classLabels = 10
X = x_train_vectors
y = y_train

def nnCostFunction(nnThetas, X, y, lambda_ = 0, inputLayerSize = 784, hiddenLayerSize = 25, classLabels = 10):
    
    m = len(X)
    
    #reshaping the list of parameters to matrices
    hiddenLayerParamNr    = (hiddenLayerSize * (inputLayerSize+1))
    thetaOneMatrix        = np.reshape(nnThetas[0:hiddenLayerParamNr],
                                       newshape = (hiddenLayerSize, inputLayerSize+1))
    outputLayerParamStart = hiddenLayerParamNr 
    thetaTwoMatrix        = np.reshape(nnThetas[outputLayerParamStart:],
                                       newshape = (classLabels, hiddenLayerSize+1))
    
    #Your turn! Implement forward propagation below, then calculate the cost for the forward_propagated samples
    #Feel free to remove the def nnCostfunction: -part and just play with the steps in this cell or a new cell below
        #and to then later assemble it back into a function
    
    return None #change this to return the cost!

# your answer


## Adding regularisation

You probably already reckoned from the inclusion of a lambda_ argument that there was going to be some regularisation. Indeed there is. We don't want to overfit our network to the training data, and it's still a good strategy to do so by adding a penalty for making too many weights too large.

The regularised cost function looks as follows:
![RegCostFunction](RegularisedCostFunctionNN.PNG) <br>

It might seem a bit daunting, but all it's saying is 'loop over every parameter for every unit in the HL and the output layer, and add the square of that number to a running sum, then scale with $\lambda$ and divide by the number of samples'. It's no different from before.

Up to you to:
* Copy your function from above in the cell below and edit it so that it can return a regularised cost.
* Once done, test it with a $\lambda$ of 100. The resultant cost should be ~7.09.

Hint:
* We don't regularise the bias!
* `np.square()` is probably your friend.
* Remember `lambda_/2*m != lambda_/(2*m)`

In [ ]:
# your answer


## Numerical gradient checking

Before we get to implementing backpropagation, there is one thing we need to do to be sure that we implement it correctly: make a function that can check this for us. In this case, we do that by numerically approximating the gradient. We do this by taking our current $\theta$ value and adding some small value $e$, and also taking our $\theta$ value and substracting this small value $e$. Then, we can calculate: <br> <br>
$$f(\theta) \approx \frac{J(\theta + e) - J(\theta - e)}{2e} $$ 

This corresponds to the following image: ![numericalApproximationGradient](NumericalGradientComputation.PNG) <br>

If this explanation leaves you unsure, Welch labs has got you covered in [this great video](https://www.youtube.com/watch?v=pHMzNW8Agq4).
Do note that the computational cost is **exorbitant**: you have to calculate the cost for all your training samples twice per parameter in the network, and our tiny network has some ~19.500 of them. 

Up to you to:
* Define a function `numericalGradientApproximation()` that takes in nnThetas, X, y, and lambda_ as arguments, and runs `nnCostFunction()` for the current theta +$e$ and -$e$, and divides them by 2$e$. Set $e$ to $1\cdot10^{-4}$ 
* **NOTE**: remember that you want to do this for each $\theta$ separately: we are calculating the partial derivative w.r.t. each parameter in the network. In other words: you should change _only one_ theta in each calculation (one time subtracting $e$ from it, one time adding $e$ to it) of the cost function and keep the rest the same. 

In [ ]:
#your answer



## Implementing backpropagation part 1: sigmoid gradient

As a first step to actually do backpropagation, your task is to implement the sigmoid gradient function in the code cell below: $sigmoid(x)*(1-sigmoid(x))$. In keeping with the age-old traditions of our people, call it `mySigmoidGradient`.

In [ ]:
#your answer



## What I want you to remember here:
* What the (regularised) cost function for neural networks is and how to implement it.
* That you can also approximate gradients numerically, but that this is _very_ computationally expensive. Prohibitively so, which is why we only use it to check ~ourselves before we wreck ourselves~.
* That the derivative of the sigmoid function is what it is.
* That [banana ketchup](https://en.wikipedia.org/wiki/Banana_ketchup) is a thing.

## The end
Well done, regularised cost function: check, sigmoid function: check, hotel: Trivago. Now it's time for convolutional neural networks and a guest talk before we move on to the afternoon practical where you'll actually implement backpropagation!

## Survey
[BAM](https://docs.google.com/forms/d/e/1FAIpQLScmJpZ3voXHZrTR4KjJbreJH5-IdIjyNBBfhf7bw0GigpmMdw/viewform?usp=sf_link)